In [16]:
import numpy as np
from keras.models import load_model
from pythainlp.tokenize import word_tokenize
from thai_tokenizer import Tokenizer
from pythainlp import word_vector
from keras.models import Model, load_model
import numpy as np

In [17]:
# Read input.txt into separate lists
with open("input.txt", "r", encoding="utf-8-sig") as input_file:
    input_data = [line.strip().split("::")[1] for line in input_file]

tokenizer = Tokenizer()

# Pythai Tokenize
words = [
    [w for w in word_tokenize(s, engine="newmm", keep_whitespace=False)]
    for s in input_data
]

print(len(words))
# Print the tokenized words
for sentence in words:
    print(sentence)

FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'

In [ ]:
vocab = set([w for s in words for w in s])
print('Vocab size = '+str(len(vocab)))
print(vocab)

Vocab size = 1122
{'เสียน้ำตา', 'ติดสินบน', 'ตาจน', 'ทั่วไป', 'กระทะ', 'ไม่', 'มีนัด', 'ตุกติก', '8', 'หวัง', 'แถม', 'ดุ', 'คู่', 'ขึ้น', 'ติดกัน', 'หลังจากที่', 'ทั้งสอง', 'เดี๋ยว', 'สะท้อน', 'เห็น', 'ใต้', 'กล้า', 'สีม่วง', 'กำไร', 'สบาย', 'แบบ', 'ถ่ายทอดสด', 'น้องสาว', 'แว่นตากันแดด', 'แผ่น', 'คนเดียว', 'ภาวะ', 'จะ', 'บ้า', 'สเปรย์', 'ฝ้าฟาง', 'ต่อหน้า', 'คณะ', 'คับ', 'ซัก', 'ถือว่า', 'ชีส', 'รอบ', 'ไกล', 'ส่วนมาก', 'นั่ง', 'หมู', 'ถูก', 'ใช้', 'ร้อน', 'สี่', 'สำคัญ', 'เป้า', 'คนดี', 'จุด', 'นั้น', 'คุณหมอ', 'ส', 'เปลี่ยน', 'ทางธรรมชาติ', 'เธอ', 'เคารพนับถือ', 'อายุ', 'เข้ากับ', 'เลือก', 'ไหน', 'เข้าสู่', 'รสชาติ', 'สาเหตุ', 'สาร', 'ถึงขั้น', 'ตาปลา', 'แก้มือ', 'ท่อ', 'ทดสอบ', 'กินข้าว', 'เฒ่า', 'เรียงแถว', 'ห้องมืด', 'เอาไว้', 'เสียง', 'ตาย', 'นี้', 'ผ่าน', 'ป่า', 'โกรธ', 'ผ้าก๊อซ', 'ลืมตัว', '79', 'ขาว', '7', 'ทำ', 'สี', 'ร่วมกับ', 'เป็นโรค', 'มัก', 'จิตใจ', 'ตาเหลือง', 'ห้องเช่า', 'ไร', 'สิ', 'ลง', 'ทุกๆ', 'สัก', 'จั่ว', 'จ้อง', 'เฉลี่ย', 'วันหลัง', 'ของขวัญ', 'ต้อง', 'ความสุข', 

In [ ]:
#Word Vector Test
word_vector_length = 300
wv = word_vector.WordVector(model_name="thai2fit_wv").get_model() # load thai2fit_wv from pythainlp

max_sentence_length = 34
print('Input Line',len(words))
print('Max Sentence Length',max_sentence_length)

input = np.zeros((len(words),max_sentence_length,word_vector_length))
sample_count = 0
for s in words[:len(words)]:
    word_count = 0
    for w in s:
        try:
            input[sample_count,max_sentence_length-word_count-1,:] = wv[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1

# print(input)
print(input.shape)

Input Line 200
Max Sentence Length 34
(200, 34, 300)


In [ ]:
class_mapping = {0: 'E', 1: 'P', 2: 'T'}

# Load the model
model = load_model('model.h5')
print(model.summary())

pred = model.predict(input)

pred_labels = []
for i in range(len(pred[0])):
    output1_argmax = pred[0][i].argmax()
    output1_label = class_mapping[output1_argmax] if output1_argmax in class_mapping else ''

    output2_argmax = pred[1][i].argmax()
    output2_label = class_mapping[output2_argmax] if output2_argmax in class_mapping else ''

    output3_argmax = pred[2][i].argmax()
    output3_label = class_mapping[output3_argmax] if output3_argmax in class_mapping else ''

    # Combine the predicted labels
    predicted_labels = [label for label in [output1_label, output2_label, output3_label] if label != '']
    predicted_labels_str = ",".join(predicted_labels)

    pred_labels.append(f"{i + 1}::{predicted_labels_str}")

# Save predictions to a text file
with open('ans.txt', 'w') as f:
    f.write("\n".join(pred_labels))


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 34, 300)]    0           []                               
                                                                                                  
 bidirectional_9 (Bidirectional  (None, 34, 256)     439296      ['input_4[0][0]']                
 )                                                                                                
                                                                                                  
 bidirectional_10 (Bidirectiona  (None, 34, 256)     394240      ['bidirectional_9[0][0]']        
 l)                                                                                               
                                                                                            